In [1]:
from features import ensembleFeature
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib as mpl
import seaborn as sns
import torch
from torch import nn
import re
from sklearn.utils import shuffle
from utils import metricsCal
from torch.utils.data import DataLoader,TensorDataset
import math
import sys
import copy
import pickle
from torch.autograd import Variable
from sklearn.model_selection import KFold
import torch.nn.functional as F
import os
import Attention_model as Model
#import Model
from matplotlib import cm
from matplotlib.ticker import LinearLocator
import csv
from itertools import combinations,permutations
from sklearn.preprocessing import StandardScaler
from time import time
from thundersvm import SVC
%matplotlib inline

In [2]:
#一个函数用于序列转换，一个函数用于检测“N”碱基
def long_short(data):
    seq_list = []
    for i in data[0]:
        seq_list.append(str(i)[480:521])
    return np.array(seq_list)

def check_N(data1,data2):
    seq_list = []
    gene_list = []
    for i in range(len(data1)):
        if str(data1[i]).find("N")<0:
            seq_list.append(data1[i])
            gene_list.append(data2[i])
    return np.array(seq_list),np.array(gene_list)

In [3]:
type_name = ["FullTranscript","matureRNA"]
cell_name = ["A549","CD8T","Hek293_abacm","Hek293_sysy","HeLa","MOLM13"]
# type_name = ["matureRNA"]
# cell_name = ["MOLM13"]
#feature_name = ["DBPF","emb","PCP","PSNP"]
feature_name=["PCP"]
network_name = ["BILSTM+Self-Attention","BILSTM+MultiSelf-Attention","BILSTM+Bah-Attention"]
#network_name = ["BILSTM+Self-Attention","BILSTM+MultiSelf-Attention","BILSTM+Bah-Attention","BILSTM+MultiHead-Attention"]

#rows = len(type_name)*len(cell_name)*len(feature_name)*len(network_name)*5
rows = len(type_name)*len(cell_name)*len(feature_name)*len(network_name)*5
row = -1
batch_size = 400
row_name = []
parameter_header_record = ["name","th","TN","FN","FP","TP","Sen","Spe","Acc","Mcc","AUC"]

Parameter = np.zeros(rows*10).reshape(rows,10)

In [4]:
Folder_name = []
for typei in range(len(type_name)):
    for celli in range(len(cell_name)):
        Folder_list = []
        trainPosSeq_file = "data/"+type_name[typei]+"/"+cell_name[celli]+"/Pos_"+cell_name[celli]+"_train_"+type_name[typei]+"_seq.csv"
        trainNegSeq_file = "data/"+type_name[typei]+"/"+cell_name[celli]+"/Neg_"+cell_name[celli]+"_train_"+type_name[typei]+"_seq.csv"
        testPosSeq_file = "data/"+type_name[typei]+"/"+cell_name[celli]+"/Pos_"+cell_name[celli]+"_test_"+type_name[typei]+"_seq.csv"
        testNegSeq_file = "data/"+type_name[typei]+"/"+cell_name[celli]+"/Neg_"+cell_name[celli]+"_test_"+type_name[typei]+"_seq.csv"
        trainPosGene_file = "data/"+type_name[typei]+"/"+cell_name[celli]+"/Pos_"+cell_name[celli]+"_train_"+type_name[typei]+"_gene.csv"
        trainNegGene_file = "data/"+type_name[typei]+"/"+cell_name[celli]+"/Neg_"+cell_name[celli]+"_train_"+type_name[typei]+"_gene.csv"
        testPosGene_file = "data/"+type_name[typei]+"/"+cell_name[celli]+"/Pos_"+cell_name[celli]+"_test_"+type_name[typei]+"_gene.csv"
        testNegGene_file = "data/"+type_name[typei]+"/"+cell_name[celli]+"/Neg_"+cell_name[celli]+"_test_"+type_name[typei]+"_gene.csv"
        
        #获取文件信息
        trainPos_seq = pd.read_csv(trainPosSeq_file,header=None)
        trainNeg_seq = pd.read_csv(trainNegSeq_file,header=None)
        testPos_seq = pd.read_csv(testPosSeq_file,header=None)
        testNeg_seq = pd.read_csv(testNegSeq_file,header=None)
        trainPos_gene = pd.read_csv(trainPosGene_file) 
        trainNeg_gene = pd.read_csv(trainNegGene_file)
        testPos_gene = pd.read_csv(testPosGene_file)
        testNeg_gene = pd.read_csv(testNegGene_file)
        #序列转41-nt长度
        trainPos_seq = long_short(trainPos_seq)
        trainNeg_seq = long_short(trainNeg_seq)
        testPos_seq = long_short(testPos_seq)
        testNeg_seq = long_short(testNeg_seq)
        #基因转35长度
        trainPos_gene = trainPos_gene.iloc[:,[0,1,3,4,5,6,7,8,9,10,11,13,14,16,17,19,23,22,25,20,21,44,45,46,47,48,49,50,65,66,69,55,51,57,56]]
        trainNeg_gene = trainNeg_gene.iloc[:,[0,1,3,4,5,6,7,8,9,10,11,13,14,16,17,19,23,22,25,20,21,44,45,46,47,48,49,50,65,66,69,55,51,57,56]]
        testPos_gene = testPos_gene.iloc[:,[0,1,3,4,5,6,7,8,9,10,11,13,14,16,17,19,23,22,25,20,21,44,45,46,47,48,49,50,65,66,69,55,51,57,56]]
        testNeg_gene = testNeg_gene.iloc[:,[0,1,3,4,5,6,7,8,9,10,11,13,14,16,17,19,23,22,25,20,21,44,45,46,47,48,49,50,65,66,69,55,51,57,56]]
        trainPos_seq,trainPos_gene = check_N(trainPos_seq,np.array(trainPos_gene))
        trainNeg_seq,trainNeg_gene = check_N(trainNeg_seq,np.array(trainNeg_gene))
        testPos_seq,testPos_gene = check_N(testPos_seq,np.array(testPos_gene))
        testNeg_seq,testNeg_gene = check_N(testNeg_seq,np.array(testNeg_gene))
        #基因变Float
        trainPos_gene = np.float32(trainPos_gene)
        trainNeg_gene = np.float32(trainNeg_gene)
        testPos_gene = np.float32(testPos_gene)
        testNeg_gene = np.float32(testNeg_gene)
        #打乱顺序，再取等长
        trainNeg_seq, trainNeg_gene = shuffle(trainNeg_seq,trainNeg_gene, random_state=1)
        testNeg_seq, testNeg_gene = shuffle(testNeg_seq, testNeg_gene, random_state=1)

        trainNeg_seq = trainNeg_seq[:len(trainPos_seq)]
        trainNeg_gene = trainNeg_gene[:len(trainPos_gene)]
        testNeg_seq = testNeg_seq[:len(testPos_seq)]
        testNeg_gene = testNeg_gene[:len(testPos_gene)]
        
        #建立空表
        header_name = ["true"]
        
        #第二重循环
        for featurei in range(len(feature_name)):
            if feature_name[featurei] == "binary":
                trainPos = ensembleFeature.Binary(trainPos_seq)
                trainNeg = ensembleFeature.Binary(trainNeg_seq)
                testPos = ensembleFeature.Binary(testPos_seq)
                testNeg = ensembleFeature.Binary(testNeg_seq)
            elif feature_name[featurei] == "DNC":
                trainPos = ensembleFeature.DNC(trainPos_seq)
                trainNeg = ensembleFeature.DNC(trainNeg_seq)
                testPos = ensembleFeature.DNC(testPos_seq)
                testNeg = ensembleFeature.DNC(testNeg_seq)
            elif feature_name[featurei] == "NCPA":
                trainPos = ensembleFeature.NCPA(trainPos_seq)
                trainNeg = ensembleFeature.NCPA(trainNeg_seq)
                testPos = ensembleFeature.NCPA(testPos_seq)
                testNeg = ensembleFeature.NCPA(testNeg_seq)
            elif feature_name[featurei] == "gene":
                trainPos = trainPos_gene
                trainNeg = trainNeg_gene
                testPos = testPos_gene
                testNeg = testNeg_gene
            elif feature_name[featurei] == "emb":
                trainPos = ensembleFeature.emb_seqs(trainPos_seq)
                trainNeg = ensembleFeature.emb_seqs(trainNeg_seq)
                testPos = ensembleFeature.emb_seqs(testPos_seq)
                testNeg = ensembleFeature.emb_seqs(testNeg_seq)
            elif feature_name[featurei] == "PSNP":
                trainPos,trainNeg,testPos,testNeg = ensembleFeature.PSNP(trainPos_seq,trainNeg_seq,testPos_seq,testNeg_seq)
            elif feature_name[featurei] == "ENAC":
                trainPos = ensembleFeature.ENAC(trainPos_seq)
                trainNeg = ensembleFeature.ENAC(trainNeg_seq)
                testPos = ensembleFeature.ENAC(testPos_seq)
                testNeg = ensembleFeature.ENAC(testNeg_seq)
            elif feature_name[featurei] == "EIIP":
                trainPos = ensembleFeature.EIIP(trainPos_seq)
                trainNeg = ensembleFeature.EIIP(trainNeg_seq)
                testPos = ensembleFeature.EIIP(testPos_seq)
                testNeg = ensembleFeature.EIIP(testNeg_seq)
            elif feature_name[featurei] == "PseDNC":
                trainPos = ensembleFeature.PseDNC(trainPos_seq)
                trainNeg = ensembleFeature.PseDNC(trainNeg_seq)
                testPos = ensembleFeature.PseDNC(testPos_seq)
                testNeg = ensembleFeature.PseDNC(testNeg_seq)
            elif feature_name[featurei] == "PCP":
                trainPos = ensembleFeature.PCP(trainPos_seq)
                trainNeg = ensembleFeature.PCP(trainNeg_seq)
                testPos = ensembleFeature.PCP(testPos_seq)
                testNeg = ensembleFeature.PCP(testNeg_seq)
            elif feature_name[featurei] == "DBPF":
                trainPos = ensembleFeature.DBPF(trainPos_seq)
                trainNeg = ensembleFeature.DBPF(trainNeg_seq)
                testPos = ensembleFeature.DBPF(testPos_seq)
                testNeg = ensembleFeature.DBPF(testNeg_seq)
            if len(testNeg.shape)==2:
                trainPos = trainPos.reshape(trainPos.shape[0],1,trainPos.shape[1])
                trainNeg = trainNeg.reshape(trainNeg.shape[0],1,trainNeg.shape[1])
                testPos = testPos.reshape(testPos.shape[0],1,testPos.shape[1])
                testNeg = testNeg.reshape(testNeg.shape[0],1,testNeg.shape[1])

            trainPos_label = np.ones(len(trainPos))
            trainNeg_label = np.zeros(len(trainNeg))
            testPos_label = np.ones(len(testPos))
            testNeg_label = np.zeros(len(testNeg))

            trainData = np.append(trainPos,trainNeg,axis=0)
            testData = np.append(testPos,testNeg,axis = 0)
            trainLabel = np.append(trainPos_label,trainNeg_label,axis = 0)
            testLabel = np.append(testPos_label,testNeg_label,axis = 0)
            
            if featurei == 0:
                result_table = testLabel.reshape(-1,1)

            kf = KFold(5,True,10)    
            #第三重循环
            for networki in range(len(network_name)):
                if network_name[networki] == "BILSTM+Self-Attention":         
                    for i,[train_index, test_index] in enumerate(kf.split(trainData)):
                        row+=1
                        row_one = type_name[typei]+"_"+cell_name[celli]+"_"+feature_name[featurei]+"_"+network_name[networki]+"_KFold_" + str(i)
                        row_name.append(row_one)
                        header_name.append(row_one)

                        X_train = trainData[train_index]
                        X_test = trainData[test_index]
                        Y_train = trainLabel[train_index]
                        Y_test = trainLabel[test_index]
                
                        model_dir = "Model_Attention_Extraction_Final/"+type_name[typei]+"_"+cell_name[celli]+"/"+\
                                feature_name[featurei]+"_"+network_name[networki]+"/KFold_" + str(i) + "/"
                    
                        model = Model.ModelBS_Pro(trainData.shape[1],trainData.shape[2])
                        
                        device = torch.device("cuda:1" if torch.cuda.is_available() else "cpu")  #选择设备
                        os.environ['CUDA_VISIBLE_DEVICES'] = '1'
                        
                        
                        y_pred,y_true = Model.independResult(X_train,Y_train,device,model_dir,batch_size,0.5)
                        Parameter[row][0],_,_,_,_,_,_,_,_,_ = metricsCal.get_train_metrics(y_pred,y_true)
                        y_pred,y_true = Model.independResult(testData,testLabel,device,model_dir,batch_size,Parameter[row][0])
                        Parameter[row][1],Parameter[row][2],Parameter[row][3],Parameter[row][4],Parameter[row][5],Parameter[row][6],Parameter[row][7],Parameter[row][8],Parameter[row][9] = metricsCal.get_test_metrics(y_pred,y_true,Parameter[row][0])
                        #y_pred,y_true = Model.independResult(testData,testLabel,device,model_dir,batch_size,0.5) 
                        result_table = np.append(result_table,y_pred.reshape(-1,1),axis = 1)
                        print(row_one)
                
                if network_name[networki] == "BILSTM+MultiSelf-Attention":         
                    for i,[train_index, test_index] in enumerate(kf.split(trainData)):
                        row+=1
                        row_one = type_name[typei]+"_"+cell_name[celli]+"_"+feature_name[featurei]+"_"+network_name[networki]+"_KFold_" + str(i)
                        row_name.append(row_one)
                        header_name.append(row_one)

                        X_train = trainData[train_index]
                        X_test = trainData[test_index]
                        Y_train = trainLabel[train_index]
                        Y_test = trainLabel[test_index]
                
                        model_dir = "Model_Attention_Extraction_Final/"+type_name[typei]+"_"+cell_name[celli]+"/"+\
                                feature_name[featurei]+"_"+network_name[networki]+"/KFold_" + str(i) + "/"
                    
                        model = Model.ModelB_MultiSelf_Pro(trainData.shape[1],trainData.shape[2])
                        
                        device = torch.device("cuda:1" if torch.cuda.is_available() else "cpu")  #选择设备
                        os.environ['CUDA_VISIBLE_DEVICES'] = '1'
                        
                        y_pred,y_true = Model.independResult(X_train,Y_train,device,model_dir,batch_size,0.5)
                        Parameter[row][0],_,_,_,_,_,_,_,_,_ = metricsCal.get_train_metrics(y_pred,y_true)
                        y_pred,y_true = Model.independResult(testData,testLabel,device,model_dir,batch_size,Parameter[row][0])
                        Parameter[row][1],Parameter[row][2],Parameter[row][3],Parameter[row][4],Parameter[row][5],Parameter[row][6],Parameter[row][7],Parameter[row][8],Parameter[row][9] = metricsCal.get_test_metrics(y_pred,y_true,Parameter[row][0])
                        #y_pred,y_true = Model.independResult(testData,testLabel,device,model_dir,batch_size,0.5) 
                        result_table = np.append(result_table,y_pred.reshape(-1,1),axis = 1)
                        print(row_one)
                        
                if network_name[networki] == "BILSTM+Bah-Attention":         
                    for i,[train_index, test_index] in enumerate(kf.split(trainData)):
                        row+=1
                        row_one = type_name[typei]+"_"+cell_name[celli]+"_"+feature_name[featurei]+"_"+network_name[networki]+"_KFold_" + str(i)
                        row_name.append(row_one)
                        header_name.append(row_one)

                        X_train = trainData[train_index]
                        X_test = trainData[test_index]
                        Y_train = trainLabel[train_index]
                        Y_test = trainLabel[test_index]
                
                        model_dir = "Model_Attention_Extraction_Final/"+type_name[typei]+"_"+cell_name[celli]+"/"+\
                                feature_name[featurei]+"_"+network_name[networki]+"/KFold_" + str(i) + "/"
                    
                        model = Model.ModelB_Bah_Pro(trainData.shape[1],trainData.shape[2])
                        
                        device = torch.device("cuda:1" if torch.cuda.is_available() else "cpu")  #选择设备
                        os.environ['CUDA_VISIBLE_DEVICES'] = '1'
                        
                        y_pred,y_true = Model.independResult(X_train,Y_train,device,model_dir,batch_size,0.5)
                        Parameter[row][0],_,_,_,_,_,_,_,_,_ = metricsCal.get_train_metrics(y_pred,y_true)
                        y_pred,y_true = Model.independResult(testData,testLabel,device,model_dir,batch_size,Parameter[row][0])
                        Parameter[row][1],Parameter[row][2],Parameter[row][3],Parameter[row][4],Parameter[row][5],Parameter[row][6],Parameter[row][7],Parameter[row][8],Parameter[row][9] = metricsCal.get_test_metrics(y_pred,y_true,Parameter[row][0])
                        #y_pred,y_true = Model.independResult(testData,testLabel,device,model_dir,batch_size,0.5) 
                        result_table = np.append(result_table,y_pred.reshape(-1,1),axis = 1)
                        print(row_one)
                if network_name[networki] == "BILSTM+MultiHead-Attention":         
                    for i,[train_index, test_index] in enumerate(kf.split(trainData)):
                        row+=1
                        row_one = type_name[typei]+"_"+cell_name[celli]+"_"+feature_name[featurei]+"_"+network_name[networki]+"_KFold_" + str(i)
                        row_name.append(row_one)
                        header_name.append(row_one)

                        X_train = trainData[train_index]
                        X_test = trainData[test_index]
                        Y_train = trainLabel[train_index]
                        Y_test = trainLabel[test_index]
                
                        model_dir = "Model_Attention_Extraction_Final/"+type_name[typei]+"_"+cell_name[celli]+"/"+\
                                feature_name[featurei]+"_"+network_name[networki]+"/KFold_" + str(i) + "/"
                    
                        model = Model.ModelB_Multi_Pro(trainData.shape[1],trainData.shape[2])
                        
                        device = torch.device("cuda:1" if torch.cuda.is_available() else "cpu")  #选择设备
                        os.environ['CUDA_VISIBLE_DEVICES'] = '1'
                        
                        y_pred,y_true = Model.independResult(X_train,Y_train,device,model_dir,batch_size,0.5)
                        Parameter[row][0],_,_,_,_,_,_,_,_,_ = metricsCal.get_train_metrics(y_pred,y_true)
                        y_pred,y_true = Model.independResult(testData,testLabel,device,model_dir,batch_size,Parameter[row][0])
                        Parameter[row][1],Parameter[row][2],Parameter[row][3],Parameter[row][4],Parameter[row][5],Parameter[row][6],Parameter[row][7],Parameter[row][8],Parameter[row][9] = metricsCal.get_test_metrics(y_pred,y_true,Parameter[row][0])
                        #y_pred,y_true = Model.independResult(testData,testLabel,device,model_dir,batch_size,0.5) 
                        result_table = np.append(result_table,y_pred.reshape(-1,1),axis = 1)
                        print(row_one)
        pd.DataFrame(result_table).to_csv("First_Second_Result/"+type_name[typei]+"_"+cell_name[celli]+"_"+feature_name[featurei]+".csv",index=0,header=header_name)

/data/xiayunpeng/anaconda3/envs/BERT/lib/python3.6/site-packages/sklearn/utils/validation.py:72: FutureWarning: Pass shuffle=True, random_state=10 as keyword args. From version 1.0 (renaming of 0.25) passing these as positional arguments will result in an error
  "will result in an error", FutureWarning)
/data/xiayunpeng/PycharmWorkspace/Whistle/Attention_model.py:56: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  emb = torch.tensor(emb) + self.pe[:,:emb.shape[1]]
/data/xiayunpeng/PycharmWorkspace/Whistle/Attention_model.py:56: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  emb = torch.tensor(emb) + self.pe[:,:emb.shape[1]]


FullTranscript_A549_PCP_BILSTM+Self-Attention_KFold_0


/data/xiayunpeng/PycharmWorkspace/Whistle/Attention_model.py:56: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  emb = torch.tensor(emb) + self.pe[:,:emb.shape[1]]
/data/xiayunpeng/PycharmWorkspace/Whistle/Attention_model.py:56: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  emb = torch.tensor(emb) + self.pe[:,:emb.shape[1]]


FullTranscript_A549_PCP_BILSTM+Self-Attention_KFold_1


/data/xiayunpeng/PycharmWorkspace/Whistle/Attention_model.py:56: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  emb = torch.tensor(emb) + self.pe[:,:emb.shape[1]]
/data/xiayunpeng/PycharmWorkspace/Whistle/Attention_model.py:56: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  emb = torch.tensor(emb) + self.pe[:,:emb.shape[1]]


...

matureRNA_MOLM13_PCP_BILSTM+Bah-Attention_KFold_2


/data/xiayunpeng/PycharmWorkspace/Whistle/Attention_model.py:56: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  emb = torch.tensor(emb) + self.pe[:,:emb.shape[1]]
/data/xiayunpeng/PycharmWorkspace/Whistle/Attention_model.py:56: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  emb = torch.tensor(emb) + self.pe[:,:emb.shape[1]]


matureRNA_MOLM13_PCP_BILSTM+Bah-Attention_KFold_3


/data/xiayunpeng/PycharmWorkspace/Whistle/Attention_model.py:56: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  emb = torch.tensor(emb) + self.pe[:,:emb.shape[1]]
/data/xiayunpeng/PycharmWorkspace/Whistle/Attention_model.py:56: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  emb = torch.tensor(emb) + self.pe[:,:emb.shape[1]]


matureRNA_MOLM13_PCP_BILSTM+Bah-Attention_KFold_4


In [6]:
Parameter[-1]

array([4.53407288e-01, 4.87100000e+03, 3.26500000e+03, 4.09700000e+03,
       5.70300000e+03, 6.35927743e-01, 5.43153434e-01, 5.89540589e-01,
       1.79856871e-01, 6.24359068e-01])

In [7]:
header_record = ["name","th","TN","FN","FP","TP","Sen","Spe","Acc","Mcc","AUC"]
pd.DataFrame(np.append(np.array(row_name).reshape(len(row_name),1),Parameter,axis=1)).to_csv("First_Second_Result/PCP_Parameter.csv",index=0,header=header_record)